In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as image_utils

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense

2024-04-20 17:52:17.179002: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 17:52:17.179076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 17:52:17.179696: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-20 17:52:17.185339: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
image_dir = './Jaws_binary_labeled_images/'


print("Class number:", len(os.listdir(train_image_dir)))

Class number: 2


In [5]:
# Input image dimensions, etc.
resized_height = 224
resized_width = 224
num_channel = 3 
num_classes = 18
batch_size = 32

In [6]:
train_images = image_dataset_from_directory(
    image_dir, labels='inferred', label_mode='categorical',
     color_mode='rgb', image_size=(resized_height ,
    resized_width ), batch_size=batch_size, shuffle=True, seed=42,
    validation_split=0.2, subset='training',
    interpolation='bilinear', follow_links=False,
    crop_to_aspect_ratio=False)

Found 108 files belonging to 2 classes.
Using 87 files for training.


2024-04-20 17:55:24.593298: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 860 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:b7:00.0, compute capability: 8.0


In [7]:
valid_images = image_dataset_from_directory(
    image_dir, labels='inferred', label_mode='categorical',
     color_mode='rgb', image_size=(resized_height ,
    resized_width ), batch_size=32, shuffle=True, seed=42,
    validation_split=0.2, subset='validation',
    interpolation='bilinear', follow_links=False,
    crop_to_aspect_ratio=False)

Found 108 files belonging to 2 classes.
Using 21 files for validation.


In [8]:
# Initialize empty lists to store images and labels
images = []
labels = []

# Iterate over the dataset to extract images and labels
for image_batch, label_batch in train_images:
    images.append(image_batch.numpy())
    labels.append(label_batch.numpy())

# Concatenate the lists of images and labels
image_array = np.concatenate(images, axis=0)
label_array = np.concatenate(labels, axis=0)

print("Images shape:", image_array.shape)  # Output: (87, 224, 224, 3)
print("Labels shape:", label_array.shape)  # Output: (87, 18)

Images shape: (87, 224, 224, 3)
Labels shape: (87, 2)


In [9]:
def custom_preprocessing_function(image):
    # Randomly select a channel to scale
    channel_to_scale = np.random.randint(0, 3)  # 0 for Red, 1 for Green, 2 for Blue

    # Generate a random scaling factor between 0.5 and 1.5
    scaling_factor = np.random.uniform(0.5, 1.5)

    # Scale the selected channel
    image[:,:,channel_to_scale] *= scaling_factor

    # Clip the values to ensure they remain in the valid range [0, 255]
    image = np.clip(image, 0, 255)

    return image


In [10]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

In [18]:
augmented_images = datagen.flow(image_array, label_array)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential([
    Conv2D(8, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPool2D((2, 2)),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    Conv2D(16, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='sigmoid')
])


# CNNs are more concise and have fewer parameteres than dense layers

In [21]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 8)       224       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 111, 111, 8)       0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 16)      1168      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 54, 54, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 52, 52, 16)        2320      
                                                                 
 flatten_2 (Flatten)         (None, 43264)            

In [19]:
# Fit the generator on the training data
datagen.fit(image_array)

# Compile the new model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
history= model.fit(augmented_images,
          epochs=20,
#          steps_per_epoch=4,
          validation_data=valid_images)

Epoch 1/20


2024-04-20 18:03:59.235121: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8800
2024-04-20 18:03:59.420687: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 82.50MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-20 18:03:59.420828: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 82.50MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-20 18:03:59.422517: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 82.50MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could

NotFoundError: Graph execution error:

Detected at node sequential_2/conv2d_7/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/apps/tensorflow/2.15/lib/python3.11/site-packages/ipykernel/__main__.py", line 5, in <module>

  File "/apps/tensorflow/2.15/lib/python3.11/site-packages/traitlets/config/application.py", line 1046, in launch_instance

  File "/apps/tensorflow/2.15/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/apps/tensorflow/2.15/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/apps/tensorflow/2.15/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/apps/tensorflow/2.15/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/apps/tensorflow/2.15/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/apps/tensorflow/2.15/lib/python3.11/site-packages/